# Spam Mail Detection
- Using Mail Data - /content/mail_data.csv
- Pre Processing ->
  - Null Missing replace column string hai
  - Label change 0/1 convert HAM SPAM ko
  - Text to numerical data TfIdfVectorizer
- Train and Test - sklearn.model_selection
- Logistic Regression
- Accuracy score -> for classification model


In [6]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

## Data collection and preprocessing

In [10]:
mail_df= pd.read_csv("/content/mail_data.csv")
mail_df.tail()

,Category,Message
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...
5571,ham,Rofl. Its true to its name


### Replacing the null values with a null string

In [12]:
mail_df.isnull().sum()

,0
Category,0
Message,0


In [13]:
mail_df.shape

(5572, 2)

In [11]:
# mail_data= mail_df.where((pd.notnull(mail_df)),'')
# OR
mail_df.fillna('', inplace=True)

### Label Encoding spam0 ham1

In [14]:
mail_df['Category'] = mail_df['Category'].replace({'spam': 0, 'ham': 1})

/tmp/ipython-input-14-3753352665.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  mail_df['Category'] = mail_df['Category'].replace({'spam': 0, 'ham': 1})


In [15]:
mail_df.head()

,Category,Message
0,1,"Go until jurong point, crazy.. Available only ..."
1,1,Ok lar... Joking wif u oni...
2,0,Free entry in 2 a wkly comp to win FA Cup fina...
3,1,U dun say so early hor... U c already then say...
4,1,"Nah I don't think he goes to usf, he lives aro..."


### Searating x and y labels

In [19]:
x= mail_df['Message']
y = mail_df['Category']

(5572,)


### Train and Testing split

In [20]:
x_train,x_test,y_train,y_test= train_test_split(x,y,test_size=0.2,random_state=3)

In [21]:
print(x_train.shape,x_test.shape)

(4457,) (1115,)


### Feature Extraction
  - using stop words remove all to lowercase
    - Helps reduce noise from uninformative words.
  - min_df=1
    - A word must appear in at least 1 document to be kept.
  - String to number can be used as input to logistic regression
  - feature vectors

In [24]:
feature_extraction= TfidfVectorizer(min_df=1,stop_words='english',lowercase=True)

In [48]:
feature_extraction.fit(x_train)
x_train_features = feature_extraction.transform(x_train)
x_test_features = feature_extraction.transform(x_test)
# FIT using x_train features

In [50]:
print(x_train_features.shape)

(4457, 7431)


In [34]:
print(y_test)
# Access the first value
first_value = y_test.iloc[0]

# Print the value
print("First value:", first_value)

# Print the type of the value
print("Type:", type(first_value))

2632    0
454     1
983     0
1282    1
4610    1
       ..
4827    1
5291    1
3325    1
3561    1
1136    1
Name: Category, Length: 1115, dtype: int64
First value: 0
Type: <class 'numpy.int64'>


#### Convert y values as integer

In [37]:
y_train = y_train.astype('int')
y_test = y_test.astype('int')
# You're trying to cast to a specific Python type (int vs np.int64)
print(type(y_train))

<class 'pandas.core.series.Series'>


## Train the model

In [40]:
model = LogisticRegression()

In [41]:
model.fit(x_train_features,y_train)

LogisticRegression()

### Predicting on training data

In [51]:
prediction_on_training = model.predict(x_train_features)
accuracy_score_on_training  = accuracy_score(prediction_on_training,y_train)

In [52]:
print(accuracy_score_on_training)

0.9676912721561588


### Predicition on test data

In [53]:
prediction_on_testing = model.predict(x_test_features)
accuracy_score_on_testing  = accuracy_score(prediction_on_testing,y_test)
print(accuracy_score_on_testing)

0.9668161434977578


## Build a predictive system

In [55]:
input_mail= ["Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile Update Co FREE on 08002986030"]
# convert to feature vector
input_feature = feature_extraction.transform(input_mail)
print(input_feature)


prediction = model.predict(input_feature)
print(prediction)

if prediction[0]==1:
  print("Mail is Ham")
else:
  print("Mail is Spam")

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10 stored elements and shape (1, 7431)>
  Coords	Values
  (0, 264)	0.26684458508660697
  (0, 1561)	0.2341600532618622
  (0, 1828)	0.25340864742428515
  (0, 2533)	0.28763638582781614
  (0, 2870)	0.32348775308028876
  (0, 3883)	0.23149865187136975
  (0, 4386)	0.3701107539734877
  (0, 4387)	0.2824854097001953
  (0, 4421)	0.300615809050912
  (0, 6907)	0.5114944868344367
[0]
